# Stream Cipher


### Stream Ciphers

A stream cipher is an encryption technique that works byte by byte to transform plain text into code that's unreadable to anyone without the proper key. It is highly efficient, and can provide real-time encryption and decryption. The main problem is making sure the keystream is unpredictable enough and not to repetative. 

In [114]:
import numpy as np
from functools import reduce
from operator import xor
from itertools import islice

## LFSR


### Linear-feedback shift register (LFSR)

LFSR is used to generate a pseudo-randrom stream of binary.
It iterates a number of times, using the uses the previous polynomial to compute the next. If often shifts one place at the time, and can make a somewhat long row of binary numbers.


In [109]:
#to be used for converting

def bin_to_int(fist_bin):
  return int("".join(str(x) for x in fist_bin), 2)

def int_to_bin(int_numb, N):
  binary = [1 if digit == '1' else 0 for digit in bin(int_numb)[2:]]
  return [0 for i in range(N - len(binary))] + binary

In [219]:
class LFSR:

    ''' Attributes:
        poly (list of int): The coefficients of the polynomial function in the LFSR.
        length (int): The length of the LFSR sequence.
        output (bool): The current output bit of the LFSR.
        feedback (bool): The current feedback bit of the LFSR.
        state (list of bits): The current state of the LFSR. '''

    def __init__(self, poly, state=None):
        ''' Initializes the LFSR object with a given polynomial and state'''

        self.length = max(poly)
        self.output = None
        self.feedback = 0

        self.poly = [0 for _ in range(max(poly) + 1)] 
        for i in range(len(self.poly)):
            if i in poly:
                self.poly[i] = 1
        self.poly.reverse()
        self.poly.pop()

        if state is None:
            self.state = [1 for _ in range(max(poly) + 1)] #all is 1 if no state
        else:
            if isinstance(state, int):
                self.state = int_to_bin(state, max(self.poly))
            else:
                self.state = [(digit) for digit in state]

        if len(self.state) < len(self.poly):
            self.state = [1 for _ in range(len(self.poly))] #in case the state is shorter than the poly

    def __iter__(self):
        return self

    def __next__(self):
        '''Calculates the next output bit of the LFSR sequence and updates the state'''
        anded = []
        for i in range(len(self.poly)):
            anded.append(self.state[i] and self.poly[i])
        self.feedback = reduce(xor, anded)
        self.output = self.state[0]
        self.state.pop(0)
        self.state.append(self.feedback)
        return self.output

    def run_steps(self, N=1):
        '''Runs the LFSR for N steps and returns a list of the output bits'''
        list_of_bool = []
        for i in range(N):
            list_of_bool.append(self.__next__())
        return list_of_bool

    def cycle(self, state=None):
        '''Runs the LFSR for a full cycle and returns the output bits.'''
        if state is not None: #uses a new state if provided
            self.state = state
        return self.run_steps(N=2**self.length - 1) #length of full cycle

    def __str__(self):
        string = ''
        for b in islice(self, 8):
            string += str(self.state) + '\n'
        return string.rstrip()


### Testing of LFSR

The testing should check different scenarios, such as the three mentioned under, to verify that the code works as expected.

Prove class functionality (test every method) in these three cases:
- `poly = [3, 1, 0]` and `state = 6` (`0b110`)
- `poly = [5, 2, 1, 0]` and `state = 5` (`0b00101`)
- `poly = [9, 5, 0]` and `state = 511` (`0b111111111`)`


In [218]:
lfsr_1 = LFSR([3, 1, 0], state=6)
print(lfsr_1)

lfsr_2 = LFSR([5, 2, 1, 0], state=5)
print(lfsr_2)

lfsr_3 = LFSR([9, 5, 0], state=511)
print(lfsr_3)

#the code runs, and more testing is done when testing BM


[1, 0, 1]
 [0, 1, 0]
 [1, 0, 0]
 [0, 0, 1]
 [0, 1, 1]
 [1, 1, 1]
 [1, 1, 0]
 [1, 0, 1]


## Berlekamp Massey


### Berlekamp Massey (BM) algorithm 

The Berlekamp-Massey algorithm is a technique that can identify the most compact LFSR capable of generating a given binary sequence. This approach takes a binary sequence as input and generates a feedback polynomial as output that describes the shortest possible LFSR that can produce the input sequence.

In [199]:
def berlekamp_massey(bits):
  N = len(bits)
  P = [0 for i in range(N)]
  P[N-1] = 1
  m = 0
  Q = [0 for i in range(N)]
  Q[N-1] = 1 
  r = 1

  for t in range(N):
    anded = []
    x_r = [0 for i in range(N)]

    #check negative indexing
    for j in range(m+1):

      P_inverted = P[::-1]
      anded.append(P_inverted[j] and bits[t-j])
      d = reduce(xor, np.array(anded))

    if d == 1: 
      if 2 * m <= t:
        R = P
        x_r[N-r-1] = 1
        Qx_r = bin_to_int(Q) * bin_to_int(x_r)
        P_int = bin_to_int(P)
        P_int = np.bitwise_xor(P_int, Qx_r)

        P = int_to_bin(P_int, N)
        Q = R
        m = t + 1 - m
        r = 0
      else: 
        x_r[N-r-1] = 1
        Qx_r = bin_to_int(Q) * bin_to_int(x_r)
        P_int = bin_to_int(P)
        P_int = np.bitwise_xor(P_int, Qx_r)

        P = int_to_bin(P_int, N)

    r = r + 1

  return P[::-1][:m+1]

def print_poly(polynomial): 
      '''Helping function to make it look'''
    if polynomial[0] == 1:
        result = '1 + '
    else: 
        result = ''
    for idx,i in enumerate(polynomial[1:]):
        if i == 1: 
            result += 'x^{}'.format(idx+1)
            if idx != (len(polynomial)-2):
                result += ' + '
    return result

### testing


### Testing BM 

For testing, we first compute the LFSR, and use at least one cycle. When we put the bitstream back, it is expected that the same polynomial is returned.

Prove its functionality in these three cases, using the already defined LFSR class:
- `poly = [3, 1, 0]` and `state = 6` (`0b110`)
- `poly = [5, 2, 1, 0]` and `state = 5` (`0b00101`)
- `poly = [9, 5, 0]` and `state = 511` (`0b111111111`)`


In [200]:
lfsr_1 = LFSR([3, 1, 0], state=6)
bitstream_1 = lfsr_1.cycle()
print(bitstream_1)
bm_poly1 = berlekamp_massey(bitstream_1)
print(f"Shortest LFSR: {print_poly(bm_poly1)}\n")

lfsr_2 = LFSR([5, 2, 1, 0], state=5)
bitstream_2 = lfsr_2.cycle()
print(bitstream_2)
bm_poly2 = berlekamp_massey(bitstream_2)
print(f"Shortest LFSR: {print_poly(bm_poly2)}\n")
#this one returns a shorter one, not sure why

lfsr_3 = LFSR([9, 5, 0], state=511)
bitstream_3 = lfsr_3.cycle()
print(bitstream_3)
bm_poly3 = berlekamp_massey(bitstream_3)
print(f"Shortest LFSR: {print_poly(bm_poly3)}\n")

[1, 1, 0, 1, 0, 0, 1]
Shortest LFSR: 1 + x^1 + x^3

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Shortest LFSR: 1 + x^1

[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1

Given the following key stream `keystream`, generated by an LFSR, apply the Berlekamp-Massey Algorithm to discover the structure (in particular, the polynomial) of the LFSR.


In [103]:
keystream = b"\xfa\xb3v\x93\x8b\xca0\x83"

# Convert keystream in a list of bits here:
def keystream_to_bits(keystream):
    bits = []
    for byte in keystream:
        bits.extend([int(bit) for bit in bin(byte)[2:].zfill(8)])
    return bits

# Run BM here:
bits = keystream_to_bits(keystream)
bm_poly = berlekamp_massey(bits)
print(print_poly(bm_poly))


1 + x^1 + x^6


## LFSR-based Generator


### the Alternating Step Generator

It alternates between different LFSR, which already are random, to make it even more secure. It uses a clock, which is either high or low, to controll the different LFSRs. 


- `poly0 = [5, 2, 0]`
- `poly1 = [3, 1, 0]`
- `polyC = [2, 1, 0]`


In [169]:
class AlternatingStepGenerator:
    ''' Generates bits using an alternating step generator (ASG) with LFSRs '''
    
    def __init__(self, seed):
        ''' Initializes the ASG with the given seed '''
        
        if seed == 0:
            raise ValueError("Seed value cannot be zero.")
        
        # Initialize the LFSRs, hardcoded 
        self.lfsr_c = LFSR([2, 1, 0], seed)
        self.lfsr_0 = LFSR([5, 2, 0], seed)
        self.lfsr_1 = LFSR([2, 1, 0], seed)

        #so they will have a start value
        self.clock_output = next(self.lfsr_c)
        self.lfsr0_output = next(self.lfsr_0)
        self.lfsr1_output = next(self.lfsr_1)
        
        
    def __iter__(self):
        return self
    
    def __next__(self):
        ''' Generates the next bit in the ASG '''
        self.clock_output = next(self.lfsr_c)
        if self.clock_output == 0:
            self.lfsr0_output = next(self.lfsr_0)
        else:
            self.lfsr1_output = next(self.lfsr_1)
        output = self.lfsr0_output^self.lfsr1_output
        return output
        



In [170]:
asg = AlternatingStepGenerator(seed=5)
bits = [next(asg) for _ in range(1000)]
print(bits)

[0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 

Compute the Linear complexity of the generated sequence.

In [172]:
# Generate a sequence of 1000 bits using the ASG
asg = AlternatingStepGenerator(seed=5)
seq = [next(asg) for i in range(100)]

# Compute the linear complexity of the generated sequence using the Berlekamp-Massey algorithm
coeffs = berlekamp_massey(seq)
linear_complexity = len(coeffs)

print(f"The linear complexity of the generated sequence is {linear_complexity}.")

The linear complexity of the generated sequence is 51.


### testing


### ASG as stream cipher

The Alternating Step Generator (ASG) can be employed to build a stream cipher by generating a stream of key bits, which can be XORed with the plaintext to generate the ciphertext. The basic idea is to use the ASG to produce a pseudorandom sequence of bits that are unpredictable to an attacker, and then use this sequence of bits as a keystream to encrypt the plaintext


Given the key `key`, decrypt the ciphertext `ciphertext`.


In [173]:
key = 0x3FF # integer value
ciphertext = (
    b'Y\xea\xfc\xc2\x8c\x17p{\x1f8\xbf,N\\\xf8\x97\xeb\x99#\'#\xf3\x1cY\xfd'
    b'\x82\xe9\xbe\xc2\xeb\x16H\xd0Q\xd5\xa8Y\x8e\x8b\\\xeb\x8d\xe1\xea\xf5'
    b'\x83\xb0\xe7\xee\xf2\\\xear\x848l\xe2\xb2\x06ov%\xdb\x08\x13\xf2qy\xf6'
    b'}\xfdO\xebG{\xaa\xaf\xbaP\xc1\x98c\x19\xdc5\xf3\x00P_\x0e\x8b\xe9\xa5$'
    b'\xaf,7\x99G\xe5\x89f\xd5o\xd9s\xd4\x10;\xf4\x10\x1a\x84]\xf6>\xd9J\x86'
    b'\xb2/%\x86\x92\xdc3f\x1d\x15\xa0\xa7K1*\xa0\xaa\x88+x\xb9\xa9#Ie)\xf97'
    b'\x05\xf1\x1b\x02~\xeac\xd0\xeb\x0bv+\xd2Y\xb1\xbd\x1d6!\xde\xd9\x9de'
    b'\x05\xb3\xf30\x14\xfd\xa7s\xa9\xce\xad]\x01W\x0b\x9a\xb6\x03\x8e^J\xbf'
    b'\x01\xe4\xb6\xa0\x83\xbej\xb6\xda\xca\xa3J\xd78~1\xaf/t\x1eL\x93\x19J'
    b'\xb18Bm\x7f\xddZ\xa8L\xbd\xb1\x84,\x04\xc7\x0b,\x11\xce}\xed\x06\x06XZ'
    b'\xaf\xcf\xaa-\xaf\xad"\x87\xd3\xf8\xc2\xf7\xd3\x1bM}\xb9\x00)\xb1\x9a'
    b'\x06\xcdU\xedV\xd7\x03\x90\xed;>\xc5\xd7\xff\xa0qZ\x94\xf3\xb6\x1b9v'
    b'\xfa\xfa\xf1x}\xd9\xf3\x7fm\xe4 \xe0"G\xe0O*C%p\xd7yYS&\xd9{\xec\xe9'
    b'=46\xfbH\xcc#\x0f\xe8\xf78H\xcc *\xb8\xd8\xe35\xda\x03>\xc5\xf0\x1a'
    b'OCZ\xfc\x11\xbd\xf7\xb0\xc9\xb2!\xfe\xd8\xc7\x8e\x1c\xc3:\x7fb\xdd9wZ'
    b'\xad\xca\\\x83\xf9>Fx\x1dQ\x1d\x9a\x92\xdb\xc1\x8b+\x19\xdfDK\x93\xd7M'
    b'\xe7Cg\xdbP\xa6\x99\xe5`\xae\xed6E\xcf\xe3\xc2\xb5\xee\x80\x14D+@5\xb2'
    b'\xde\x02\xdb\x01\x9b\xd9\x90<\x00\xe6\n=\x98\xf6\xe9\xb7\x14\x93\x95'
    b'\xc8\xf7YX!\xe2\x830<q\x9b\xed\x034\xa0\x0c|(\x05%h3\x87dN\x160zN\''
    b'\x8ev\xe4\xe0\xb0q\x02\xb1\x10\xa0\x90\x06\xf42SSV4nl\xf4\xd8\xe1\xc3S'
    b'?\x89\xe5\x80\x11X\x1f\xfe-"\xed\xb4D\xb6a\xa3\xdd\xc8\xca\t\xfcrg\x0e'
    b'\xfa|X\x16\x82\xc2\xdb\x86\xfd=\x07cK\x15?\x98\xd3\xf8\xda\xcb\x0c\x0e'
    b'\x84\\\x9c\x84\x87\xd1\xa5P\xab\xcd;'
)


In [175]:
def asg_decrypt(key, ciphertext):
    asg = AlternatingStepGenerator(key)
    plaintext = bytearray()
    for byte in ciphertext:
        plaintext.append(byte ^ next(asg))
    return plaintext

print(asg_decrypt(key=key, ciphertext=ciphertext))

#didnt manage to do this:/

bytearray(b'Y\xea\xfc\xc2\x8c\x17p{\x1f8\xbf,N\\\xf8\x97\xeb\x99"\'#\xf3\x1cY\xfd\x82\xe8\xbf\xc3\xea\x17H\xd0Q\xd5\xa9X\x8f\x8a]\xea\x8c\xe0\xea\xf4\x82\xb1\xe6\xef\xf3]\xear\x849l\xe3\xb3\x06nw$\xda\t\x12\xf3qy\xf6}\xfdN\xeaFz\xaa\xaf\xbaQ\xc1\x98c\x18\xdd4\xf2\x01Q^\x0f\x8b\xe9\xa5%\xaf-7\x99F\xe5\x88f\xd5n\xd9r\xd5\x11;\xf4\x10\x1b\x84]\xf7>\xd9J\x86\xb2.%\x87\x93\xdd2g\x1c\x14\xa1\xa7K1*\xa0\xaa\x88+x\xb8\xa9"Ie)\xf97\x04\xf0\x1a\x02~\xeac\xd1\xea\x0bw*\xd3Y\xb1\xbc\x1d6 \xdf\xd8\x9dd\x04\xb3\xf21\x14\xfc\xa6r\xa9\xce\xac\\\x01V\n\x9a\xb6\x02\x8e^K\xbe\x00\xe4\xb7\xa1\x82\xbej\xb7\xda\xcb\xa2K\xd69\x7f0\xaf.u\x1fL\x92\x19K\xb19Bl~\xddZ\xa8L\xbd\xb1\x84,\x05\xc6\n-\x11\xce}\xec\x07\x07X[\xae\xcf\xab,\xae\xac#\x87\xd3\xf9\xc3\xf7\xd2\x1bL}\xb9\x01(\xb1\x9a\x07\xccT\xecV\xd6\x03\x90\xec;>\xc5\xd6\xff\xa0qZ\x94\xf3\xb6\x1a9w\xfb\xfb\xf0x}\xd9\xf2\x7fm\xe5 \xe0"G\xe0N+B$q\xd7xYS&\xd9z\xec\xe9<56\xfaH\xcc"\x0e\xe8\xf78I\xcc!*\xb8\xd9\xe24\xdb\x03?\xc5\xf0\x1bOBZ\xfd\x11\xbc\xf6\xb0\xc9\

In [165]:
key = 0x3FF
asg = AlternatingStepGenerator(key)

# Let's generate key stream bytes until we have enough to XOR with the ciphertext
keystream = bytes()
while len(keystream) < len(ciphertext):
    keystream += bytes([next(asg)])

# XOR the ciphertext with the keystream to get the plaintext
plaintext = bytes([ciphertext[i] ^ keystream[i] for i in range(len(ciphertext))])
print(plaintext)

b'\xaf\x16sE\xab\x07\x7fC\xf2\xf7{#\xb0\xbc\x86\xe6\xf3\x0b\x1a\xe0\xdf\x858\xb6J\xa5\x987"j\xd8W\xb3\xa9\xf6\xd7\xa8\x95\x15R\xef\x10\x90\x15z\x02\x8f\x9af\x0b\xbd\x92w\xe6\xf7\x94\x9d\xf3\xdc\x91x\xe2J\xc84|F\x9e\x86s\x07\xae\x95Zu\xe3\x96\xc8\xa6\xdb\x0b\\\xc6M\xc5\x17\xe3P\xb8\x132\x10$\xdbs"\xf9\xbaT\xa3\x91\x99[\xaaE\x850\xfe\x0b\xc9\x00"f\xb0\x89;\xb7\xb5\xa5\xc1\xee\xe4\x15\x1d\xc3\xc2\x97;\xfb\x10\x99W>\n,\xd2V%\xb1\x83\xd7\xe3\x90\x96\x10\xfc\xc7\xc2\r\xf4\x03\x80\x97l3j\xc2q%\xac\xba\xc3\xfa\xc7\xc5\x1e\xc3E\xddY\xe6\x04\xcc!*\x07&\x8al\xad\xe4\xbf\x1e\xa6\xd3\x08\x08\xdf\x12\x99m=\x07\xd9\xcb.b2\x94s\xb5\xb2\x9bX\xa7\xfa\x81\x0f\xab\xd3\x838\xae\x13\xfcZ?\x03\xa2\x94x\xb2\xa3\x92S\xfb\xb0\x1a\x15\xda[\xca\x13\xfeO\x80\x9d?&\xb9\x94s\xa0\xe3\xd7\xd0\xeb\xe0\x15[\xff_\x804\xf9K\x83\xc9?\x11=\xd2\x1a\xae\xab\xd6E\xa1\xd8\x02T\xff\xce\x840p\x11\x8cJ+\n;\xd5?u\xb5\xc2\x8f\xbe\xbdFM\xbb\x90\x94\x1cb\x13\xa0\x9cp\x0b\xaf\xdbv\xb5\xb0\xb8A\xe0\xc2\x88T\xee\xc1\xc5\tuN\xe8?6\x13z\xf

## RC4


### Rivest Cipher 4 (RC4)

RC4 is a symmetric key stream cipher algorithm. It operates on a variable-length key and produces a keystream used for encryption and decryption. RC4 generates the keystream by iterating through an internal state that is initialized based on the key. The keystream is generated using a pseudo-random number generator, which generates a sequence of random numbers that are combined with the plaintext using bitwise XOR operation to produce ciphertext. Decryption is achieved by generating the same keystream using the same key and then XORing it with the ciphertext to produce the plaintext.

In [106]:
class RC4():
    
    def __init__(self, seed, drop=0):
        ''' Initialize the RC4 cipher with a given seed and drop value '''
        self.S = list(range(256))
        j = 0
        for i in range(256):
            j = (j + self.S[i] + seed[i % len(seed)]) % 256
            self.S[i], self.S[j] = self.S[j], self.S[i]
        self.i = self.j = 0
        for _ in range(drop):
            self.__next__()

    def __iter__(self):
        return self
    
    def __next__(self):
        ''' Generate the next byte in the keystream '''
        self.i = (self.i + 1) % 256
        self.j = (self.j + self.S[self.i]) % 256
        self.S[self.i], self.S[self.j] = self.S[self.j], self.S[self.i]
        return self.S[(self.S[self.i] + self.S[self.j]) % 256]

### Testing RCA


#### RC4 employed to build a stream cipher

RC4 can be employed to build a stream cipher by using its keystream generator. The keystream is generated by the RC4 algorithm, which generates a sequence of pseudo-random bytes based on a secret key. This keystream can then be combined with the plaintext using the XOR operation to produce the ciphertext.

To encrypt a message, the sender generates a unique key and initializes the RC4 algorithm with that key. The sender then generates the keystream by repeatedly calling the RC4 algorithm. The keystream is XOR-ed with the plaintext to produce the ciphertext, which is then transmitted to the receiver. The receiver uses the same key to initialize their own RC4 algorithm and generates the same keystream, which is XOR-ed with the ciphertext to recover the plaintext.

Given the key `key`, decrypt the ciphertext `ciphertext`.


In [107]:
key = b"0123456789ABCDEF"  # integer value
ciphertext = (
    b"\x0f\x9e\xec\xc3\x0f\xeck\xa5\xc9\xd4\xd4\xb81\xf0\xd5[\x93"
    b"\xbc\xdaY^\x0e\xc1'\xdb\x1eP\xc0uD\xeb\x91E]\x15\xcf\x85\x07%"
    b"\x97\xffl\xf1\xbb\xe1*\xa0\xee\xd3\x94\x17\xb0|e\x93h\xd7\xc5"
    b"\xc3tT\x84\xee%\xcb\x15\r\xc4\xe8\n\xf2\xd0.\xec\xc6\xe1-\xb4"
    b"\x8b\x9a\x14\x823k\xab?\x8b\x9c\xaas\xa1#\xb8\xb2\xceh\xb5\x8b"
    b"'\x90B}C\x80~\x8cr\xde\xc9\xe2\x17\xe45\xb9\x10\x94\xd4\x0eRJ"
    b"\x0fr&\xe7\xe3P\xbfz\xecIA\x94\xe60\xa8{_\x03\xc7\x91\xcf\xc6"
    b"\x04\xfc\x8d\x86-E\x13\xba\x13i\x17;\xd7\x8e \xa5\xe6\xa5uR\n"
    b"89z\xe2YZ!e\x0f,s\x9a\xacN\xab\xc7\xcaOO\x81\xe06\x03\xac7\x9b"
    b",%\xf7\x9d(\xde\x0b\xc3\xbf\xbe7\xc7<\xf4r\x0eLz\xd8\xe5b\xa0n"
    b"\x11f\xfe\xca+RNnL\xef+\x1b\x1c\x0b\x8a\xb6M\xbdU\xd6\x1c\x9cn"
    b"\t\x8aX>B9\xa4\xf7\xd7jS\xa3\xfd\xeb=\xeeY\xbf\x8dG\xab\xc6"
    b"\x08 \xfba\x90\xdbla\xbf\xf0\x9c\x1a\xa0\xcb\x9a\x7f\x88X\x1dV"
    b"\xd2'\xba\x1b\x16l\xc3vx\xddqU\xf72@\x86>A\xe6b\x050Y\xed\x8bJ"
    b"!\xe0\x80I\xa4X\x9e\xd6^\x126\x99\x93\xa3(\xc5\xf0\xbd\xdei\xcb"
    b"\xdc5\xf58\xda\xa5\x96\xb9!5>\xdbJ\xd4\xef\xe1\x0e\xfeo\x97`"
    b"\xd7[\x18\t\xf4/DV\xdd\xe8~#\xbd\xfd{B\xacC=\xa9\xd5\xbb\xdbH"
)
drop = 3072


In [108]:
def rc4_decrypt(key, ciphertext, drop):
    rc4 = RC4(key, drop)
    plaintext = bytearray()
    for byte in ciphertext:
        plaintext.append(byte ^ next(rc4))
    return plaintext

print(rc4_decrypt(key=key, ciphertext=ciphertext, drop=drop))


bytearray(b'Hey, Macklemore, can we go thrift shopping?What? What? What? What?\nWhat? What? What? What?\nWhat? What? What? What?\nWhat? What? What? What?\nWhat? What? What? What?\nWhat? What? What? What?\nWhat? What? What? What?\nWhat? What? What? What?\nOh!\nOh!\nOw!\nI m gonna pop some keys\nOnly got 3072 drop in my RC4\nI m, I m, I m hunting, looking for a come up\nThis is ******* awesome\n')
